This notebook contains some functions from the [utils folder](utils/__init__.py) which you can use to clean, convert, transform and scale the data.\
Included will be the function documentation, how to import the function (relative to the project root), and a small example for any notable function.\
Make sure the [requirements](requirements.txt) are installed into your current python environment (see [here](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/) for more information).

# Loading the data

In [1]:
from utils.data_cleaning import load_and_clean

This [function](utils/data_cleaning.py#L8) cleans the raw data by dropping unnecessary columns and rows and converting the target price column to a float.

**Parameters:**
- *drop_columns (bool):* Whether to drop any columns.
- *drop_rows (bool):* Whether to drop any rows. It will however, always convert the target price column to a float for the sake of visualization.
- *\*\*kwargs:* Additional keyword arguments to pass to the drop_features and handle_price functions.

**Returns:**
- *pd.DataFrame*: The cleaned data as a pandas DataFrame.

---

Aditional arguments which can be passed through the *\*\*kwargs* argument, must be key=value pairs (**e.g.:** 'quantile=0.8, verbose=True'; **not:** '0.8, True'):
- *quantile: float = 0.99*, the quantile to use for outlier removal.
- *verbose: bool = False*, whether to print some meaningfull information during computation.
- *remove_redundant_feats: bool = True*, whether to remove redundant features, specified in *manual_redundant_feats* and *add_redundant_feats*.
- *manual_redundant_feats: list = [...]*, a list of features to remove. A predefined list is the default value, for exactly which features are removed, see the [function](utils/data_cleaning.py#L73).
- *add_redundant_feats: list = []*, optional list of features to add to the *manual_redundant_feats* list.
- *remove_high_corr_feats: bool = True*, whether to remove features with a high correlation to other features.
- *corr_threshold: float = 0.8*, the threshold for the correlation coefficient above which features are removed.
- *remove_missing_feats: bool = True*, whether to remove features with more than *missing_threshold* missing values.
- *missing_threshold: float = 0.5*, the threshold for the percentage of missing values above which features are removed.
- *remove_single_value_feats: bool = True*, whether to remove features with only one unique value.
- *remove_text_feats: bool = True*, whether to remove features with text values.

In [2]:
# Load all the data as is (no cleaning or removal), useful for plotting the data before and after cleaning
data = load_and_clean(False, False)

# Load the data, with different thresholds and columns to remove
data = load_and_clean(add_redundant_feats=['host_location', 'latitude'], corr_threshold=0.7, missing_threshold=.4)

# Print which columns / how many rows are removed during cleaning
data = load_and_clean(verbose=True)

Features dropped due to redundancy:
['calendar_last_scraped', 'description', 'host_about', 'host_id', 'host_location', 'host_name', 'host_neighbourhood', 'host_picture_url', 'host_thumbnail_url', 'host_url', 'id', 'last_scraped', 'listing_url', 'name', 'neighborhood_overview', 'picture_url', 'scrape_id', 'source']

Features dropped due to high correlation (>0.8):
['host_total_listings_count', 'minimum_minimum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_60', 'availability_90', 'review_scores_accuracy', 'review_scores_value', 'calculated_host_listings_count_entire_homes']

Features dropped due to amount of missing values (>50.0%):
['neighbourhood', 'neighbourhood_group_cleansed', 'calendar_updated', 'license']

Features dropped due to only one unique value:
['has_availability']

Features dropped due to being text:
['bathrooms_text']

Number of outliers removed above quantile 0.99 ($1077.0): 626
number of rows removed due to missing price: 32367
number of ro

# Data Preprocessing

In [3]:
from utils.pipeline import create_pipeline

In [4]:
# The code below is used to remove warnings from the notebook output, you can ignore it
import warnings
warnings.filterwarnings('ignore')

This [function](utils/pipeline.py#L18) creates a Sci-kit Learn pipeline for preprocessing the given dataframe. It uses information on the feature types specified [here](utils/_feature_types.py) to determine the preprocessing steps to be applied per feature.

**Parameters:**
- *df (pd.DataFrame):* The dataframe to preprocess.
- *model (BaseEstimator)*: The model to be used in the pipeline. Default is DecisionTreeRegressor(random_state=24).
- *convert (bool):* Whether to convert the features or not. Default is True.
- *impute (bool):* Whether to impute the missing values or not. Default is True.
- *encode (bool):* Whether to encode the categorical features or not. Default is True.
- *scale (bool):* Whether to scale the features or not. Default is True.
- *model_selection (bool):* Whether to add an estimator model or not. If True, an estimator model should be provided in *model*. Default is True.
    
**Returns:**
- *Pipeline:* The pipeline for preprocessing the dataframe.

---

The function can both be used to preprocess a dataframe in order to visualize the data after cleaning, or to preprocess the data and attach it to a model.\
When the *model_selection* parameter is set to True, the function will return a pipeline which can be used to fit data to a model (*.fit()* / *.predict()* / *.fit_predict()*).\
When the *model_selection* parameter is set to False, the function will return a pipeline which can be used to fit and transform the data (*.fit()* / *.transform()* / *.fit_transform()*).

The pipeline is constructed from both Sci-kit Learn native transformers, as well as custom converters and encoders. For reference or additional declarations, you can find the custom converters [here](utils/_custom_converters.py) and the custom encoders [here](utils/_custom_encoders.py).

In [5]:
# Split the data into features and target variable
# In practice, you would typically split the data into training and testing sets, but for illustration purposes, we'll use the entire dataset
X = data.drop(columns=['price'])
y = data['price']

# Preprocess the data to only convert features into interpretable values
# This will convert specific columns and return the transformed data as a pandas DataFrame
pipe = create_pipeline(data, impute=False, encode=False, scale=False, model_selection=False)
pipe.fit_transform(X)

# Preprocess the data to convert, impute, and encode all features into numeric instances
# This function requires fit_transform() to be called since it requires fitted data for target encoding
# The returned pandas DataFrame will contain the fully transformed dataset
pipe = create_pipeline(data, scale=False, model_selection=False)
pipe.fit_transform(X, y)

# Create the full pipeline to fit data to the model, with an interactive graphical interface
# As the function incorporates a model, we can only call fit() on the pipeline
pipe = create_pipeline(data) # Optional: specify the model to use as second parameter, import this model seperately from the sklearn library
pipe.fit(X, y)

Pipeline(steps=[('converters',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('convert_percentages',
                                                  PercentageConverter(),
                                                  ['host_response_rate',
                                                   'host_acceptance_rate']),
                                                 ('convert_lists',
                                                  ListConverter(),
                                                  ['host_verifications',
                                                   'amenities']),
                                                 ('convert_booleans',
                                                  BooleanConverter(),
                                                  ['host_is_superhost',
                                                   'host_has_profile_pic',
                                                   'host_identity_verif...
                                                  MultiLabelOneHotEncoder(),
                                                  ['host_verifications']),
                                                 ('encode_high_cardinality_list',
                                                  MultiLabelTargetEncoder(),
                                                  ['amenities'])],
                                   verbose_feature_names_out=False)),
                ('scalers',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  slice(None, None, None))],
                                   verbose_feature_names_out=False)),
                ('model_selection', DecisionTreeRegressor(random_state=24))])

The fitted pipeline can be used to make predictions on new data and compute metrics (note this can only be done when the function argument *model_selection* is set to *True*).

You can find a full list of the available metrics [here](https://scikit-learn.org/stable/modules/model_evaluation.html).

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Normally, you would use the test set for making predictions, but in this example we'll use the training set for simplicity
# Due to training and predicting on the same set, we are overfitting and getting a near perfect score, which is not realistic in practice
y_pred = pipe.predict(X)
print(f"""
      Values of Target 'price':
      {'-' * 80}
      predicted y values:
      {y_pred[:10]}

      actual y values:
      {y.tolist()[:10]}



      Model Performance Metrics:
      {'-' * 80}
      {'mean squared error:': <30} {mean_squared_error(y, y_pred):.5f}
      {'mean absolute error:': <30} {mean_absolute_error(y, y_pred):.5f}
      {'r2 score:': <30} {r2_score(y, y_pred):.5f}
      """)


      Values of Target 'price':
      --------------------------------------------------------------------------------
      predicted y values:
      [ 89. 220.  88.  75.  53. 120.  40. 135. 391.  83.]

      actual y values:
      [89.0, 220.0, 88.0, 75.0, 53.0, 120.0, 40.0, 135.0, 391.0, 83.0]



      Model Performance Metrics:
      --------------------------------------------------------------------------------
      mean squared error:            0.23389
      mean absolute error:           0.00418
      r2 score:                      0.99999
      
